In [21]:
import pandas as pd

co2 = pd.read_excel("data/co2.xlsx", sheet_name="Baumaterialien Matériaux1",header=0)

In [25]:
co2 = co2.dropna().reset_index().drop(columns=["index", "level_0"])
co2["Travaux de préparation"] = co2["Travaux de préparation"].astype(str)

co2.loc[92]


UBP_Total                                                                                     59300.0
UPB_Fabrication                                                                               55400.0
UBP_Élimination                                                                                3960.0
REN_Total                                                                                       114.0
REN_Total fabrication                                                                           114.0
REN_Valorisé sous forme énergétique fabrication                                                  67.3
REN_Valorisé sous forme de matière fabrication                                                   47.0
REN_Élimination                                                                                0.0221
NREN_Total                                                                                      139.0
NREN_Total fabrication                                                            

In [26]:
import pandas as pd
from deep_translator import GoogleTranslator


# Function to translate text using Google Translate
def translate_text(text, src_lang='fr', dest_lang='en'):
    return GoogleTranslator(source=src_lang, target=dest_lang).translate(text) 


# Translate the DataFrame
a = co2["Travaux de préparation"].apply(translate_text)


In [33]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

embeddings = model.encode(a)


In [34]:
uni_mat_ma = pd.read_excel("data/Uniclass2015_Ma_v1_1.xlsx", header=2)

uni_mat_ma = pd.read_excel("data/Uniclass2015_Ma_v1_1.xlsx", header=2)
embeddings_uni_ma = model.encode(uni_mat_ma.Title)

In [35]:
codes = pd.DataFrame(pd.concat([uni_mat_ma.Title], axis=0))
codes = codes.reset_index(drop=True)
codes.columns = ["raw"]

similarity = cosine_similarity(embeddings, embeddings_uni_ma)

num_top_indices = 1 

top_indices = [np.argmax(row) for row in similarity]


threshold = 0.40  # Set your desired threshold value

# Set values to "nan" if cosine similarity is below the threshold
raw_values = [codes['raw'].iloc[idx] if sim[idx] > threshold else np.nan for idx, sim in zip(top_indices, similarity)]

# Add these as new columns to your DataFrame
co2['material_emb'] = raw_values


In [36]:
co2.to_csv("kbob_mat.csv")